<a href="https://colab.research.google.com/github/jcandane/CI_Theory/blob/main/Ham_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyscf
import numpy as np
from pyscf import scf, gto, mcscf, mp, dft
import pyscf
import inspect

np.set_printoptions(precision=4, linewidth=200, threshold=2000, suppress=True)

In [2]:
mol = gto.M(atom = 'H 0 0 0; F 0 0 1.1', basis = 'ccpvdz')
mff = scf.UHF(mol).run()
# freeze 2 core orbitals
#pt = mp.MP2(mff).set(frozen = 2).run()
# freeze 2 core orbitals and 3 high lying unoccupied orbitals
#pt.set(frozen = [0,1,16,17,18]).run()
mff_hcore = mff.get_hcore()
H0_1   = (mff.mol).intor("int1e_kin")
H0_1  += (mff.mol).intor("int1e_nuc")

np.allclose(H0_1, mff_hcore)
#H0_1 - mff_hcore

converged SCF energy = -99.9873974403486  <S^2> = 5.4001248e-13  2S+1 = 1


True

In [3]:
mol = gto.M(atom='H 0 0 0; H 0 0 1.1')

nao = mol.nao_nr()
dm = np.random.random((2,nao,nao))

grids = dft.gen_grid.Grids(mol)
grids.coords = np.random.random((100,3))  # 100 random points
grids.weights = np.random.random(100)


ni = dft.numint.NumInt()
nelec, exc, vxc = ni.nr_uks(mol, grids, 'lda,vwn', dm)

In [4]:
inspect.getmembers(mff)

WARN: Attribute .damp_factor will be removed in PySCF v1.1. It is replaced by attribute .damp
WARN: Attribute .hf_energy will be removed in PySCF v1.1. It is replaced by attribute .e_tot
WARN: Attribute .level_shift_factor will be removed in PySCF v1.1. It is replaced by attribute .level_shift


[('CASCI', None),
 ('CASSCF', None),
 ('DFMP2', None),
 ('DIIS', pyscf.scf.diis.CDIIS),
 ('MP2',
  <bound method class_as_method.<locals>.fn of <pyscf.scf.uhf.UHF object at 0x7f68103ee750>>),
 ('__call__',
  <bound method StreamObject.set of <pyscf.scf.uhf.UHF object at 0x7f68103ee750>>),
 ('__class__', pyscf.scf.uhf.UHF),
 ('__delattr__',
  <method-wrapper '__delattr__' of UHF object at 0x7f68103ee750>),
 ('__dict__',
  {'_chkfile': <tempfile._TemporaryFileWrapper at 0x7f6827883850>,
   '_eri': array([0.8973, 0.5047, 0.3   , ..., 0.    , 0.    , 0.9849]),
   '_keys': {'DIIS',
    '_chkfile',
    '_eri',
    'callback',
    'chkfile',
    'conv_check',
    'conv_tol',
    'conv_tol_grad',
    'converged',
    'damp',
    'diis',
    'diis_file',
    'diis_space',
    'diis_space_rollback',
    'diis_start_cycle',
    'direct_scf',
    'direct_scf_tol',
    'e_tot',
    'init_guess',
    'init_guess_breaksym',
    'level_shift',
    'max_cycle',
    'max_memory',
    'mo_coeff',
    'mo

In [38]:
mol_hf = gto.M(atom = 'H 0 0 0; F 0 0 1.1', basis = 'ccpvdz')
mf = dft.UKS(mol_hf)
mf.xc = 'lda,vwn' # default
#mf_hf = mf_hf.newton() # second-order algortihm
mf.kernel()

Ca, Cb = (mf).mo_coeff
dipole = (mf.mol).intor("int1e_r")
H1     = (mf.mol).intor("int1e_kin")
H1    += (mf.mol).intor("int1e_nuc")
H2     = (mf.mol).intor("int2e") ###!! dont calculate!!
DA_mo, DB_mo = (mf).mo_occ
Da_mo = np.diag( DA_mo )
Db_mo = np.diag( DB_mo )
F_sαβ = mf.get_fock()
D_sαβ = mf.make_rdm1()

F_sαβ.shape

#FF = np.einsum("spq", H1) + 

mf_hcore = mf.get_hcore()
print(mf_hcore.shape)
print("==========")

def eval_xc(xc_code, rho, *args, **kwargs):
    exc = 0.01 * rho**2
    vrho = 0.01 * 2 * rho
    vxc = (vrho, None, None, None)
    fxc = None  # 2nd order functional derivative
    kxc = None  # 3rd order functional derivative
    return exc, vxc, fxc, kxc


#print("==========")
mf.get_ovlp()
v_eff = mf.get_veff()
H0_1   = (mff.mol).intor("int1e_kin")
H0_1  += (mff.mol).intor("int1e_nuc")*1.0
np.allclose(H0_1, mf_hcore)
print( np.allclose(H0_1, v_eff) )


print( mf.energy_tot() )
print( mf.e_tot ) 
#mf.define_xc_(mf._numint, eval_xc, xctype='LDA')

mf._numint

inspect.getmembers(mf)

converged SCF energy = -99.7624175432948  <S^2> = 1.9220181e-12  2S+1 = 1
(19, 19)
False
-99.7624175432948
-99.76241754329482


WARN: Attribute .damp_factor will be removed in PySCF v1.1. It is replaced by attribute .damp
WARN: Attribute .hf_energy will be removed in PySCF v1.1. It is replaced by attribute .e_tot
WARN: Attribute .level_shift_factor will be removed in PySCF v1.1. It is replaced by attribute .level_shift


[('CASCI', None),
 ('CASSCF', None),
 ('CCSD', <bound method CCSD of <pyscf.dft.uks.UKS object at 0x7f68063d9e90>>),
 ('CISD',
  <bound method class_as_method.<locals>.fn of <pyscf.dft.uks.UKS object at 0x7f68063d9e90>>),
 ('COSX',
  <bound method sgx_fit of <pyscf.dft.uks.UKS object at 0x7f68063d9e90>>),
 ('DDCOSMO',
  <bound method ddcosmo_for_scf of <pyscf.dft.uks.UKS object at 0x7f68063d9e90>>),
 ('DFMP2', None),
 ('DIIS', pyscf.scf.diis.CDIIS),
 ('Gradients',
  <bound method class_as_method.<locals>.fn of <pyscf.dft.uks.UKS object at 0x7f68063d9e90>>),
 ('Hessian',
  <bound method class_as_method.<locals>.fn of <pyscf.dft.uks.UKS object at 0x7f68063d9e90>>),
 ('MP2',
  <bound method class_as_method.<locals>.fn of <pyscf.dft.uks.UKS object at 0x7f68063d9e90>>),
 ('QCISD',
  <bound method QCISD of <pyscf.dft.uks.UKS object at 0x7f68063d9e90>>),
 ('QMMM',
  <bound method add_mm_charges of <pyscf.dft.uks.UKS object at 0x7f68063d9e90>>),
 ('TDA',
  <bound method class_as_method.<locals

In [39]:
mol = gto.M(atom='O 0 0 0; H 0 0 1; H 0 1 0', basis='ccpvdz')
mf = dft.UKS(mol)
#mf.define_xc_(mf._numint, '.2*HF + .08*LDA + .72*B88, .81*LYP + .19*VWN')
mf.kernel()
#    -76.3783361189611
#define_xc_(mf._numint, 'LDA*.08 + .72*B88 + .2*HF, .81*LYP + .19*VWN')
# -76.3783361189611
def eval_xc(xc_code, rho, *args, **kwargs):
    exc = 0.01 * rho**2
    vrho = 0.01 * 2 * rho
    vxc = (vrho, None, None, None)
    fxc = None  # 2nd order functional derivative
    kxc = None  # 3rd order functional derivative
    return exc, vxc, fxc, kxc
#mf.define_xc_(mf._numint, eval_xc, xctype='LDA')
#mf.kernel()

converged SCF energy = -75.8511902095262  <S^2> = 2.0250468e-13  2S+1 = 1


In [40]:
vxc.shape

(2, 2)

In [41]:
dm = mf.make_rdm1()
ni = dft.numint.NumInt()
nelec, exc, vxc = ni.nr_uks(mol, mf.grids, 'lda,vwn', dm)

F_sAB  = mf.get_fock()
FF_AB  = mf.get_hcore()[None,:,:] + vxc + np.einsum("ABCD, CD -> AB", mol.intor('int2e_sph', aosym=1), dm[0] + dm[1], optimize=True)[None,:,:] #- np.einsum("ADCB, CD -> AB", mol.intor('int2e_sph', aosym=1), dm[0], optimize=True)
FFF_AB = mf.get_hcore()[None,:,:] + mf.get_veff()[None,:,:] #- np.einsum("ADCB, CD -> AB", mol.intor('int2e_sph', aosym=1), dm[0], optimize=True)


#print(F_sAB[0,0])
#print(FF_AB[0])

print( np.allclose(F_sAB, FF_AB) )
print( np.allclose(F_sAB, FFF_AB) )


True
True


In [9]:
mol = gto.M(atom='H 0 0 0; H 0 0 1.1')
grids = dft.gen_grid.Grids(mol)
grids.coords = np.random.random((100,3))  # 100 random points
grids.weights = np.random.random(100)
nao = mol.nao_nr()
dm = np.random.random((nao,nao))
ni = dft.numint.NumInt()
nelec, exc, vxc = ni.nr_rks(mol, grids, 'lda,vwn', dm)

In [50]:
from pyscf import ao2mo

def get_SO_H1(scf, ξ=None, GAS="AA", supp=None):
    """
    GIVEN:  mf (PySCF Mean-Field (MF) Object, e.g. HF/KS)
            AO-Hamiltonian
            ξ (GAS vector, Generalized Active Space, active MOs)
            GAS (GAS String)
    GET:    SO-Hamiltonian
    Idea: from mf -> AO Hamiltonian, MO-coefficients 
          -> dress AO Ham with MO coefficients
    """
    ## get AO Hamiltonian
    Ha_ao = 1.*scf.get_hcore()
    Hb_ao = 1.*scf.get_hcore()
    try:
        scf.xc
        ni = dft.numint.NumInt()
        nelec, exc, vxc = ni.nr_uks(scf.mol, scf.grids, scf.xc, scf.make_rdm1())
        Ha_ao += vxc[0]
        Hb_ao += vxc[1]
    except:
        None

    if supp is not None:
        Ha_ao += supp
        Hb_ao += supp

    ## get MO Coefficients
    Ca, Cb = (scf).mo_coeff
    if ξ is None:
          ξ  = np.arange(0, len(Ca), 1, dtype=int)
    o = np.arange(0, len(Ca), 1, dtype=int)
    χ = np.delete(o, ξ)

    if GAS[0] == "I":
        a = χ
    else:
        a = ξ
    if GAS[1] == "I":
        b = χ
    else:
        b = ξ

    return np.array([ Ca[:,a].T @ Ha_ao @ Ca[:,b] , Cb[:,a].T @ Hb_ao @ Cb[:,b] ])


def get_SO_H2(uhf_pyscf, ξ=None, GAS="AAAA"):
    """
    GIVEN:  mf (PySCF Mean-Field (MF) Object, e.g. HF/KS)
            AO-Hamiltonian
            ξ (GAS vector, Generalized Active Space, active MOs)
            GAS (GAS String)
    GET:    SO-Hamiltonian
    Idea: from mf -> AO Hamiltonian (ERI), MO-coefficients 
          -> dress AO Ham with MO coefficients
    """
    ## get MO Coefficients
    Ca, Cb = (uhf_pyscf).mo_coeff
    
    if ξ is None:
        ξ  = np.arange(0, len(Ca), 1, dtype=int)
    o = np.arange(0, len(Ca), 1, dtype=int)
    χ = np.delete(o, ξ)

    if GAS[0] == "I":
        a = χ
    else:
        a = ξ
    if GAS[1] == "I":
        b = χ
    else:
        b = ξ
    if GAS[2] == "I":
        c = χ
    else:
        c = ξ
    if GAS[3] == "I":
        d = χ
    else:
        d = ξ

    HF_EXCHANGE = True
    try:
        uhf_pyscf.xc
        HF_EXCHANGE = False
    except:
        None
    
    aa = a.shape[0]
    bb = b.shape[0]
    cc = c.shape[0]
    dd = d.shape[0]
    eri_aa = (ao2mo.general( (uhf_pyscf)._eri , (Ca[:,a], Ca[:,b], Ca[:,c], Ca[:,d]), compact=False)).reshape((aa,bb,cc,dd), order="C")
    eri_bb = (ao2mo.general( (uhf_pyscf)._eri , (Cb[:,a], Cb[:,b], Cb[:,c], Cb[:,d]), compact=False)).reshape((aa,bb,cc,dd), order="C")
    if np.array_equal(a, b) and np.array_equal(c, d) and np.array_equal(a, d) and HF_EXCHANGE:
        print("sdfsdfsdf")
        eri_aa -= eri_aa.swapaxes(1,3)
        eri_bb -= eri_bb.swapaxes(1,3)
    eri_ab = (ao2mo.general( (uhf_pyscf)._eri , (Ca[:,a], Ca[:,b], Cb[:,c], Cb[:,d]), compact=False)).reshape((aa,bb,cc,dd), order="C")
    #if np.array_equal(a, b) and np.array_equal(c, d) and np.array_equal(a, d) and HF_EXCHANGE:
    #eri_ba = (1.*eri_ab).swapaxes(0,3).swapaxes(1,2) ## !! caution depends on symmetry
    eri_ba = (ao2mo.general( (uhf_pyscf)._eri , (Cb[:,a], Cb[:,b], Ca[:,c], Ca[:,d]), compact=False)).reshape((aa,bb,cc,dd), order="C")
    return np.stack(( np.stack((eri_aa, eri_ab)), np.stack((eri_ba, eri_bb)) ))

In [51]:
#'mo_energy', 'mo_occ'
scff = mf
da, db = scff.mo_occ
D_mo_srt = np.array([ np.diag(da), np.diag(db) ])

np.allclose( (get_SO_H1(scff) + np.einsum("STpqrs, Trs -> Spq", get_SO_H2(scff), D_mo_srt))[0] , np.diag(scff.mo_energy[0]) )

print( np.diag( (get_SO_H1(scff) + np.einsum("STpqrs, Trs -> Spq", get_SO_H2(scff), D_mo_srt))[0] ) - scff.mo_energy[0] )



[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
